In [5]:
import os

# pip install python-dotenv
from dotenv import load_dotenv

load_dotenv()

TMDB_API_KEY = os.getenv('TMDB_API_KEY')
TMDB_API_TOKEN = os.getenv('TMDB_API_TOKEN')

if TMDB_API_KEY and TMDB_API_TOKEN:
    print('✅API KEY and TOKEN are set!')
else:
    print('❌API KEY and TOKEN 404')

✅API KEY and TOKEN are set!


In [ ]:
# -*- coding: utf-8 -*-
"""
한국에서 제작 + 한국 극장 개봉(2|3) 영화만 수집 → CSV 저장(진행률 표시)
기간: 2005-01-01 ~ 2025-06-30
"""

import os, time, datetime as dt
import requests
import pandas as pd
from tqdm import tqdm
from dotenv import load_dotenv

# ===================== 설정 =====================
START_DATE = "2005-01-01"
END_DATE   = "2025-06-30"

LANG = "ko-KR"
BASE = "https://api.themoviedb.org/3"
SLEEP = 0.05                       # 요청 간 딜레이
TIMEOUT = 30

INCLUDE_ADULT = False              # 성인물 포함하려면 True
INCLUDE_VIDEO = False              # 보통 False 권장
VOTE_COUNT_MIN = 10                # 너무 마이너 작품 제거(원하면 20/50 등으로 조정)

OUT_CSV = "data/processed/tmdb_kr_theatrical_2005_2025.csv"

# ===================== 준비 =====================
load_dotenv()
API_KEY = os.getenv("TMDB_API_KEY")
if not API_KEY:
    raise SystemExit("❌ TMDB_API_KEY가 환경변수(.env)에 없습니다.")

os.makedirs(os.path.dirname(OUT_CSV), exist_ok=True)

# ===================== 공통 유틸 =====================
def get_json(url: str, params: dict, tries: int = 3):
    for i in range(tries):
        try:
            r = requests.get(url, params=params, timeout=TIMEOUT)
            if r.status_code in (429, 500, 502, 503, 504):
                time.sleep(min(2**i, 8)); continue
            r.raise_for_status()
            return r.json()
        except requests.RequestException:
            if i == tries - 1:
                raise
            time.sleep(min(2**i, 8))
    raise RuntimeError("Unreachable")

def common_params():
    return {
        "api_key": API_KEY,
        "language": LANG,
        "include_adult": str(INCLUDE_ADULT).lower(),
        "include_video": str(INCLUDE_VIDEO).lower(),
        "with_vote_count.gte": VOTE_COUNT_MIN,
        "with_origin_country": "KR",     # 🇰🇷 제작국 한국
        "region": "KR",                  # 🇰🇷 개봉 지역 한국
        "with_release_type": "2|3",      # 극장 개봉(제한/정식)
        "sort_by": "release_date.asc",   # 한국 개봉일 기준 정렬
        # release_date.gte/lte는 호출 시 주입
    }

# ===================== 1) ID 수집 =====================
def discover_total_pages(date_gte: str, date_lte: str) -> int:
    p = common_params()
    p.update({"release_date.gte": date_gte, "release_date.lte": date_lte, "page": 1})
    js = get_json(f"{BASE}/discover/movie", p)
    return int(js.get("total_pages", 1)), int(js.get("total_results", 0)), js.get("results", [])

def discover_ids_for_range(date_gte: str, date_lte: str, pbar=None):
    """범위를 /discover로 수집. total_pages<=500이면 한 번에, 아니면 호출자가 슬라이스."""
    p = common_params()
    p.update({"release_date.gte": date_gte, "release_date.lte": date_lte, "page": 1})
    first = get_json(f"{BASE}/discover/movie", p)
    total_pages = int(first.get("total_pages", 1))
    ids = [m["id"] for m in first.get("results", [])]
    if pbar: pbar.update(1)

    for page in range(2, min(total_pages, 500) + 1):
        p["page"] = page
        js = get_json(f"{BASE}/discover/movie", p)
        ids.extend([m["id"] for m in js.get("results", [])])
        if pbar: pbar.update(1)
        time.sleep(SLEEP)
    capped = total_pages > 500
    return ids, capped, total_pages

def year_slices(start: str, end: str):
    s = dt.date.fromisoformat(start); e = dt.date.fromisoformat(end)
    for y in range(s.year, e.year + 1):
        y0 = dt.date(y, 1, 1); y1 = dt.date(y, 12, 31)
        if y0 < s: y0 = s
        if y1 > e: y1 = e
        yield y0.isoformat(), y1.isoformat()

def collect_all_ids():
    print("→ ID 수집 범위:", START_DATE, "~", END_DATE)
    tp, tr, first_results = discover_total_pages(START_DATE, END_DATE)
    print(f"   예상 총 페이지: {tp} (총 편수 예상: {tr})")

    if tp <= 500:
        # 전 구간을 한 번에 수집 (페이지 기반 진행바)
        ids = [m["id"] for m in first_results]
        with tqdm(total=tp, desc="Discover pages (all range)", leave=False) as bar:
            bar.update(1)  # page=1 처리 반영
            more_ids, _, _ = discover_ids_for_range(START_DATE, END_DATE, pbar=bar)
            ids = more_ids  # discover_ids_for_range 안에서 page1부터 다시 처리했으므로 그걸 사용
        return sorted(set(ids))

    # 500 초과 → 연 단위 슬라이스 수집 (연 전체 페이지 합산해서 진행바 표시)
    # 먼저 각 연도의 페이지 수를 파악해 합산
    total_pages_sum = 0
    year_meta = []
    for yg, yl in year_slices(START_DATE, END_DATE):
        pages, _, _ = discover_total_pages(yg, yl)
        year_meta.append((yg, yl, pages))
        total_pages_sum += min(pages, 500)
    print(f"   연 단위 슬라이스 진행 (총 페이지 합산: {total_pages_sum})")

    all_ids = set()
    with tqdm(total=total_pages_sum, desc="Discover pages (by year)", leave=False) as bar:
        for yg, yl, pages in year_meta:
            ids_y, _, _ = discover_ids_for_range(yg, yl, pbar=bar)
            all_ids.update(ids_y)
            time.sleep(SLEEP)
    return sorted(all_ids)

# ===================== 2) 상세 수집 =====================
def fetch_detail(mid: int):
    params = {"api_key": API_KEY, "language": LANG}
    try:
        js = get_json(f"{BASE}/movie/{mid}", params)
        time.sleep(SLEEP)
        return js
    except requests.RequestException:
        return None

# ===================== 3) 정규화 → CSV =====================
def parse_genres(val):
    return [x.get("name") for x in val] if isinstance(val, list) else []

def normalize_rows(rows):
    df = pd.json_normalize(rows)

    out = pd.DataFrame({
        "movie_id": df.get("id"),
        "title": df.get("title"),
        "original_title": df.get("original_title"),
        "original_language": df.get("original_language"),
        "release_date": df.get("release_date"),
        "runtime": pd.to_numeric(df.get("runtime"), errors="coerce"),
        "budget": pd.to_numeric(df.get("budget"), errors="coerce"),
        "revenue": pd.to_numeric(df.get("revenue"), errors="coerce"),
        "vote_average": pd.to_numeric(df.get("vote_average"), errors="coerce"),
        "vote_count": pd.to_numeric(df.get("vote_count"), errors="coerce"),
        "popularity": pd.to_numeric(df.get("popularity"), errors="coerce"),
        "genres": df.get("genres").apply(parse_genres) if "genres" in df else [],
        "production_companies": df.get("production_companies").apply(
            lambda xs: [x.get("name") for x in xs] if isinstance(xs, list) else []
        ) if "production_companies" in df else [],
        "production_countries": df.get("production_countries").apply(
            lambda xs: [x.get("iso_3166_1") for x in xs] if isinstance(xs, list) else []
        ) if "production_countries" in df else [],
    })
    dt_series = pd.to_datetime(out["release_date"], errors="coerce")
    out["release_year"]  = dt_series.dt.year
    out["release_month"] = dt_series.dt.month
    return out

# ===================== 메인 =====================
def main():
    print("1) ID 수집 (🇰🇷 제작 ∩ 🇰🇷 극장 2|3)…")
    ids = collect_all_ids()
    print(f"   → 고유 ID 수: {len(ids):,}")

    print("2) 상세 수집 중…")
    rows = []
    with tqdm(total=len(ids), desc="Fetch details", leave=False) as bar:
        for mid in ids:
            d = fetch_detail(mid)
            if d: rows.append(d)
            bar.update(1)

    print("3) 정규화 및 CSV 저장…")
    df = normalize_rows(rows)
    df.to_csv(OUT_CSV, index=False, encoding="utf-8-sig")
    print(f"✅ 완료: {OUT_CSV} (rows={len(df):,})")

if __name__ == "__main__":
    main()


1) ID 수집 (🇰🇷 제작 ∩ 🇰🇷 극장 2|3)…
→ ID 수집 범위: 2005-01-01 ~ 2025-06-30
   예상 총 페이지: 234 (총 편수 예상: 4670)


   → 고유 ID 수: 4,670
2) 상세 수집 중…


3) 정규화 및 CSV 저장…
✅ 완료: data/processed/tmdb_kr_theatrical_2005_2025.csv (rows=4,670)


In [8]:
# -*- coding: utf-8 -*-
"""
2005-01-01 ~ 2025-12-31
KOFIC(KOBIS)에서 '한국 제작 + 국내 극장 개봉' 영화만 후보 단계부터 선별하여
TMDB 스타일 스키마 CSV 생성 (revenue=국내 최종 누적매출, audience_total=최종 누적관객)

출력 열 순서:
movie_id,title,original_title,original_language,release_date,runtime,budget,
revenue,vote_average,vote_count,popularity,genres,production_companies,
production_countries,release_year,release_month,audience_total
"""

import os, time, json, requests
import pandas as pd
import numpy as np
from datetime import datetime, timedelta, date
from urllib.parse import urlencode
from tqdm import tqdm
from dotenv import load_dotenv

# ===== 사용자 설정 =====
START_DATE = "2005-01-01"
END_DATE   = "2025-12-31"

OUT_DIR        = "./data_processed"
CANDIDATES_CSV = f"{OUT_DIR}/kofic_candidates_kr_2005_2025.csv"   # 한국만 담긴 후보
OUT_CSV        = f"{OUT_DIR}/kofic_domestic_kr_2005_2025.csv"     # 최종 결과

TIMEOUT = 20
RETRY   = 3
SLEEP   = 0.15  # 너무 낮추면 429 위험

# 주간 박스오피스 기반 최종 누적 추정 파라미터
JUMP_STEPS_DAYS     = [0, 14, 28, 56, 84]  # 개봉 후 0/2/4/8/12주
LOCAL_REFINE_RADIUS = 7                    # 마지막 앵커 ±1주 보정
MIN_ABS_DELTA  = 50_000_000               # 매출 증가 최소치(원)
MIN_REL_DELTA  = 0.01                     # 증가율 임계(1%)
PLATEAU_STREAK = 2                        # 2회 연속 정체 → 종료

# ===== 준비 =====
load_dotenv()
KOFIC_KEY = os.getenv("KOFIC_API_KEY") or os.getenv("KOBIS_KEY")
if not KOFIC_KEY:
    raise SystemExit("❌ .env에 KOFIC_API_KEY(=KOBIS_KEY)가 없습니다.")
os.makedirs(OUT_DIR, exist_ok=True)

BASE = "https://www.kobis.or.kr/kobisopenapi/webservice/rest"

def kobis_get(path, params, retry=RETRY):
    q = {"key": KOFIC_KEY, **params}
    url = f"{BASE}{path}.json?{urlencode(q)}"
    last_err = None
    for i in range(retry):
        try:
            r = requests.get(url, timeout=TIMEOUT)
            if r.status_code in (429, 500, 502, 503, 504):
                time.sleep(min(2**i, 8)); continue
            r.raise_for_status()
            data = r.json()
            if isinstance(data, dict) and data.get("faultInfo"):
                fi  = data["faultInfo"]
                raise RuntimeError(f"KOBIS fault: {fi.get('message')} (code={fi.get('errorCode')})\nURL: {url}")
            time.sleep(SLEEP)
            return data
        except Exception as e:
            last_err = e
            if i == retry - 1:
                print("❌ Request failed:", e, "\nURL:", url)
                raise
            time.sleep(min(2**i, 8))
    raise RuntimeError(f"unreachable: {last_err}")

# ===== 공통 유틸 =====
def to_iso_from_yyyymmdd(s):
    if not s: return pd.NA
    s = str(s)
    if len(s) != 8 or not s.isdigit(): return pd.NA
    return f"{s[:4]}-{s[4:6]}-{s[6:]}"

def json_list_str(xs):
    if xs is None: return "[]"
    if isinstance(xs, list):
        return json.dumps(xs, ensure_ascii=False)
    return json.dumps([str(xs)], ensure_ascii=False)

def sunday_of_week(d: date) -> date:
    # 월=0..일=6 → 일요일로 보정
    return d + timedelta(days=(6 - d.weekday()))

def is_korean_production(info):
    nations = [n.get("nationNm") for n in (info.get("nations") or []) if n.get("nationNm")]
    return any(str(n).strip() in {"한국","대한민국","Korea","South Korea","Republic of Korea"} for n in nations)

def in_period_open(info, start=START_DATE, end=END_DATE):
    openDt = info.get("openDt")
    if not openDt: return False
    iso = to_iso_from_yyyymmdd(openDt)
    if pd.isna(iso): return False
    return (iso >= start) and (iso <= end)

# ===== (옵션2) 공통코드에서 ‘한국’ 8자리 코드 조회 =====
# 에러 메시지에 ‘공통코드220310’이 언급되는 경우가 있어 다양한 후보 comCode를 순회 탐색
CODE_TABLE_CANDIDATES = ["220310", "2204", "22003", "2203", "220301", "220201"]

def get_korea_code(verbose=True):
    for cc in CODE_TABLE_CANDIDATES:
        try:
            js = kobis_get("/code/searchCodeList", {"comCode": cc})
        except Exception as e:
            if verbose:
                print(f"  · 공통코드 {cc} 조회 실패: {e}")
            continue

        # 응답 구조 가변 대응
        code_list = (
            (js.get("codeListResult") or {}).get("codeList")
            or js.get("codes")
            or js.get("codeList")
            or []
        )

        found = None
        for c in code_list:
            name_ko = c.get("korNm") or c.get("codeNm") or c.get("korName") or ""
            name_en = (c.get("engNm") or c.get("engName") or "").upper()
            code    = c.get("fullCd") or c.get("code") or c.get("cd") or ""
            if (("한국" in name_ko) or ("대한민국" in name_ko)
                or ("KOREA" in name_en) or ("REPUBLIC OF KOREA" in name_en)):
                if isinstance(code, str) and len(code) == 8 and code.isdigit():
                    found = code; break

        if found:
            if verbose:
                print(f"✅ 한국 코드 발견 (comCode={cc}): {found}")
            return found

    if verbose:
        print("⚠️ 공통코드 테이블에서 한국 코드(8자리)를 찾지 못했습니다. 응답 필터 폴백을 사용합니다.")
    return None

# ===== 1) 후보(연도별) 수집: ‘서버 필터(한국)’ 우선, 실패시 폴백(응답에서 한국만 선별) =====
KR_LABELS = {"한국","대한민국","Korea","South Korea","Republic of Korea"}

def find_candidates_by_year(year: int, kr_code: str | None):
    y = str(year)
    # 우선 ‘연/기간’ 3가지 패턴으로 프로브 → 성공 조합으로 페이징
    base_patterns = [
        {"openStartDt": y,          "openEndDt": y          },
        {"openStartDt": y                                  },
        {"openStartDt": f"{y}0101", "openEndDt": f"{y}1231"},
    ]

    def probe(params):
        js = kobis_get("/movie/searchMovieList", {**params, "itemPerPage": 10, "curPage": 1})
        res = js.get("movieListResult", {}) or {}
        tot = res.get("totCnt")
        return int(tot) if (isinstance(tot, (int,str)) and str(tot).isdigit()) else 0

    chosen = None
    mode = "ANY"

    # 1) repNationCd=한국코드 사용 프로브
    if kr_code:
        for p in base_patterns:
            params = {**p, "repNationCd": kr_code}
            if probe(params) > 0:
                chosen = params
                mode = "KCODE"
                break

    # 2) 실패 시 무필터로 프로브
    if chosen is None:
        for p in base_patterns:
            if probe(p) > 0:
                chosen = p
                mode = "ANY"
                break

    if chosen is None:
        print(f"  · {year}: 후보 0건 (건너뜀)")
        return []

    out, per_page, cur_page, total_seen = [], 100, 1, None
    while True:
        params = {**chosen, "itemPerPage": per_page, "curPage": cur_page}
        js = kobis_get("/movie/searchMovieList", params)
        res = js.get("movieListResult", {}) or {}
        lst = res.get("movieList", []) or []
        tot = res.get("totCnt", None)
        if total_seen is None and tot is not None:
            total_seen = int(tot)

        if not lst: break

        for x in lst:
            mc  = x.get("movieCd")
            rep = (x.get("repNationNm") or "").strip()
            if not mc:
                continue
            if (mode == "KCODE") or (rep in KR_LABELS):
                out.append((mc, x.get("movieNm"), x.get("openDt")))

        # repNationCd를 쓴 경우에는 totCnt에 신뢰
        if (mode == "KCODE") and (total_seen is not None) and (len(out) >= total_seen):
            break

        cur_page += 1
        time.sleep(SLEEP)

    print(f"  · {year}: mode={mode:<6}, collected(K-only)={len(out):>5}")
    return out

def build_candidates(save_path=CANDIDATES_CSV, start=START_DATE, end=END_DATE):
    s_year, e_year = int(start[:4]), int(end[:4])
    print("연도별 후보 수집(서버 한국코드 우선)…")
    kr_code = get_korea_code(verbose=True)
    cands = []
    for y in range(s_year, e_year+1):
        cands.extend(find_candidates_by_year(y, kr_code))
    df = pd.DataFrame(cands, columns=["movieCd","movieNm","openDt"]).drop_duplicates("movieCd")
    df.to_csv(save_path, index=False, encoding="utf-8-sig")
    print(f"📄 후보 저장(한국만): {save_path} (고유 {len(df):,}편)")
    return df

# ===== 2) 주간 박스오피스 기반 누적(매출/관객) 추정 =====
weekly_cache = {}

def weekly_items(target_yyyymmdd: str):
    if target_yyyymmdd in weekly_cache:
        return weekly_cache[target_yyyymmdd]
    js = kobis_get("/boxoffice/searchWeeklyBoxOfficeList",
                   {"targetDt": target_yyyymmdd, "weekGb": "0"})
    items = js.get("boxOfficeResult", {}).get("weeklyBoxOfficeList", []) or []
    weekly_cache[target_yyyymmdd] = items
    return items

def get_acc_from_weekly_by_code(movieCd: str, target_date: date):
    tgt = sunday_of_week(target_date).strftime("%Y%m%d")
    for it in weekly_items(tgt):
        if it.get("movieCd") == movieCd:
            s = pd.to_numeric(it.get("salesAcc"), errors="coerce") or 0
            a = pd.to_numeric(it.get("audiAcc"),  errors="coerce") or 0
            return s, a
    return 0, 0

def intelligent_final_acc(movieCd: str, open_yyyymmdd: str):
    if not open_yyyymmdd or len(str(open_yyyymmdd)) != 8:
        return (pd.NA, pd.NA)
    base = datetime.strptime(open_yyyymmdd, "%Y%m%d").date()

    best_sales, best_audi = 0.0, 0.0
    plateau_cnt, prev = 0, 0.0

    for dd in JUMP_STEPS_DAYS:
        anchor = base + timedelta(days=dd)
        s, a = get_acc_from_weekly_by_code(movieCd, anchor)
        if s > best_sales: best_sales = s
        if a > best_audi:  best_audi  = a

        delta = best_sales - prev
        rel = (delta/prev) if prev > 0 else 1.0
        plateau_cnt = plateau_cnt + 1 if (delta < MIN_ABS_DELTA and rel < MIN_REL_DELTA) else 0
        prev = best_sales

        if plateau_cnt >= PLATEAU_STREAK:
            for d in range(-LOCAL_REFINE_RADIUS, LOCAL_REFINE_RADIUS+1, 7):
                anchor2 = anchor + timedelta(days=d)
                s2, a2 = get_acc_from_weekly_by_code(movieCd, anchor2)
                if s2 > best_sales: best_sales = s2
                if a2 > best_audi:  best_audi  = a2
            break

    return (pd.NA if best_sales == 0 else best_sales,
            pd.NA if best_audi  == 0 else best_audi)

# ===== 3) 상세 → TMDB 스타일 행 빌드 =====
def fetch_movie_info(movieCd: str):
    js = kobis_get("/movie/searchMovieInfo", {"movieCd": movieCd})
    return js.get("movieInfoResult", {}).get("movieInfo", {}) or {}

def to_tmdb_row(info, salesAcc_final, audiAcc_final):
    movieCd   = info.get("movieCd")
    movieNm   = info.get("movieNm")
    movieNmEn = info.get("movieNmEn") or movieNm
    openDt    = info.get("openDt")  # yyyymmdd
    release_date = to_iso_from_yyyymmdd(openDt)

    dt = pd.to_datetime(release_date, errors="coerce") if pd.notna(release_date) else pd.NaT
    release_year  = (dt.year  if pd.notna(dt) else pd.NA)
    release_month = (dt.month if pd.notna(dt) else pd.NA)

    showTm = pd.to_numeric(info.get("showTm"), errors="coerce")
    genres    = [g.get("genreNm")   for g in (info.get("genres")   or []) if g.get("genreNm")]
    companies = [c.get("companyNm") for c in (info.get("companys") or []) if c.get("companyNm")]

    return {
        "movie_id": movieCd,
        "title": movieNm,
        "original_title": movieNmEn,
        "original_language": "ko",
        "release_date": release_date,
        "runtime": showTm,
        "budget": pd.NA,
        "revenue": pd.to_numeric(salesAcc_final),  # 국내 최종 누적매출(원)
        "vote_average": pd.NA,
        "vote_count": pd.NA,
        "popularity": pd.NA,
        "genres": json_list_str(genres),
        "production_companies": json_list_str(companies),
        "production_countries": json_list_str(["KR"]),
        "release_year": release_year,
        "release_month": release_month,
        "audience_total": pd.to_numeric(audiAcc_final),  # 국내 최종 누적관객
    }

def enrich_and_save(candidates_csv=CANDIDATES_CSV, out_csv=OUT_CSV):
    # 후보 로드 or 생성
    if os.path.exists(candidates_csv):
        cand = pd.read_csv(candidates_csv, dtype=str)
        print(f"📄 후보 로드: {candidates_csv} (rows={len(cand):,})")
    else:
        cand = build_candidates()

    rows = []
    with tqdm(total=len(cand), desc="Detail + domestic totals") as bar:
        for _, r in cand.iterrows():
            movieCd = str(r["movieCd"])
            try:
                info = fetch_movie_info(movieCd)
            except Exception:
                bar.update(1); continue

            # 상세에서도 한국 제작 & 기간 내 개봉 확인 (이중 안전망)
            if (not is_korean_production(info)) or (not in_period_open(info)):
                bar.update(1); continue

            try:
                s_final, a_final = intelligent_final_acc(movieCd, info.get("openDt"))
            except Exception:
                s_final, a_final = (pd.NA, pd.NA)

            rows.append(to_tmdb_row(info, s_final, a_final))
            bar.update(1)

    df = pd.DataFrame(rows)
    if not df.empty:
        df["runtime"]        = pd.to_numeric(df["runtime"], errors="coerce")
        df["revenue"]        = pd.to_numeric(df["revenue"], errors="coerce")
        df["audience_total"] = pd.to_numeric(df["audience_total"], errors="coerce")
        df["release_year"]   = pd.to_numeric(df["release_year"], errors="coerce").astype("Int64")
        df["release_month"]  = pd.to_numeric(df["release_month"], errors="coerce").astype("Int64")

    cols = ["movie_id","title","original_title","original_language","release_date","runtime","budget",
            "revenue","vote_average","vote_count","popularity","genres","production_companies",
            "production_countries","release_year","release_month","audience_total"]
    df = df.reindex(columns=cols)

    df.to_csv(out_csv, index=False, encoding="utf-8-sig")
    print(f"✅ 저장 완료: {out_csv} (rows={len(df):,})")
    print(f"🗃 weekly_cache 크기(앵커 호출 수): {len(weekly_cache):,}")

# ===== 실행 =====
if __name__ == "__main__":
    if not os.path.exists(CANDIDATES_CSV):
        build_candidates()
    enrich_and_save()


📄 후보 로드: ./data_processed/kofic_candidates_kr_2005_2025.csv (rows=2,100)


Detail + domestic totals:   0%|          | 0/2100 [00:03<?, ?it/s]


KeyboardInterrupt: 

In [3]:
year = 2019
print("↳ quick probe:", year)

# 1) repNationCd=K + YYYYMMDD
print("K + YYYYMMDD:")
print(kobis_get("/movie/searchMovieList", {
    "openStartDt": f"{year}0101", "openEndDt": f"{year}1231",
    "itemPerPage": 10, "curPage": 1, "repNationCd": "K"
}).get("movieListResult", {}).get("totCnt"))

# 2) repNationCd 제거(비교)
print("no K (compare):")
print(kobis_get("/movie/searchMovieList", {
    "openStartDt": f"{year}0101", "openEndDt": f"{year}1231",
    "itemPerPage": 10, "curPage": 1
}).get("movieListResult", {}).get("totCnt"))


↳ quick probe: 2019
K + YYYYMMDD:
None
no K (compare):
None


In [6]:
test = kobis_get("/movie/searchMovieList", {"openStartDt":"2019", "repNationCd":"K", "itemPerPage":5, "curPage":1})
for m in (test.get("movieListResult", {}).get("movieList") or []):
    print(m.get("movieNm"), m.get("repNationNm"), m.get("openDt"))
